In [30]:
!git clone https://github.com/Jhoon001/Catastro
%cd /content/Catastro/modelo/
data_path = 'dataset'
import os
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')

train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)

train_classes

import json

with open('model/index_to_name.json', 'w') as fp:
    json.dump(train_classes, fp)

test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)

test_classes

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

torch.cuda.empty_cache()

from torchvision import transforms as T

train_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

from torchvision.datasets import ImageFolder

train_dataset = ImageFolder(
    root=train_data_path,
    transform=train_transform
)

train_dataset

val_dataset = ImageFolder(
    root=train_data_path,
    transform=val_transform
)

val_dataset

test_dataset = ImageFolder(
    root=test_data_path,
    transform=val_transform
)

test_dataset

import numpy as np

val_size = .2

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(val_size * num_train))

random_seed = 42

np.random.seed(random_seed)
np.random.shuffle(indices)

from torch.utils.data.sampler import SubsetRandomSampler

train_idx, val_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    sampler=train_sampler
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    sampler=val_sampler
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=False
)
from torchvision import models

model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

import torch.nn as nn

n_inputs = model.fc.in_features
n_outputs = 10

sequential_layers = nn.Sequential(
    nn.Linear(n_inputs, 128),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.Linear(128, n_outputs),
    nn.LogSoftmax(dim=1)
)

model.fc = sequential_layers

model = model.to('cuda')

import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=.1)

loaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

dataset_sizes = {
    'train': len(train_dataset),
    'val': len(val_dataset),
    'test': len(test_dataset)
}

from copy import deepcopy

%%time

EPOCHS = 15

for epoch in range(1, EPOCHS+1):
    best_acc = .0
    print(f"\nEpoch {epoch}/{EPOCHS}\n{'='*25}")
    for phase in ['train', 'val']:
        running_loss = .0
        running_corrects = .0
        if phase == 'train': model.train()
        if phase == 'val': model.eval()
        for inputs, labels in loaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        if phase == 'train': scheduler.step()
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_weights = deepcopy(model.state_dict())
        print(f"Loss ({phase}): {epoch_loss}, Acc ({phase}): {epoch_acc}")

torch.save(best_model_weights, '../model/foodnet_resnet18.pth')


Cloning into 'Catastro'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 112 (delta 4), reused 0 (delta 0), pack-reused 91
Receiving objects: 100% (112/112), 389.05 MiB | 25.35 MiB/s, done.
Resolving deltas: 100% (14/14), done.
Updating files: 100% (45/45), done.
/content/Catastro/modelo
dataset/train
dataset/test


FileNotFoundError: Found no valid file for the classes Construcciones, Via, Zona_verde. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp